## Pré-Processamento dos Dados


Os dados contidos em datasets ou dataframes podem ser provenientes de diferentes fontes ou apresentar falhas e estar organizados de forma inadequada, resultando em análises falhas e modelos preditivos imprecisos. Sendo assim, para evitar que isso aconteça é preciso que os dados passem por uma etapa chamada de Pré-processamento. O Pré-processamento compreende um conjunto de atividades para preparação, organização e estruturação dos dados, de forma a corrigir inconsistências, agregar valor aos dados, e chegar a resultados de qualidade.

Para entender essa etapa basta imaginar o processo de cozinhar uma refeição, que inicia pela preparação dos ingredientes, e envolve ações como lavar, cortar, medir e separar os itens antes de começar a cozinhar. Então, assim como a preparação dos ingredientes, a etapa de pré-processamento também envolve a limpeza dos dados, a remoção de valores desnecessários, e a organização dos dados em "medidas" corretas. Logo, para realizar um processamento e tratamento adequado dos dados, a etapa de pré-processamento se divide na limpeza, transformação e redução dos dados.

A limpeza dos dados é responsável pelo preenchimento de dados ausentes, redução de ruídos, identificação e remoção de valores irregulares e pela resolução de inconsistências. Então, assim como você lava os vegetais, remove cascas e partes indesejadas antes de cozinhar, na limpeza de dados você remove valores irregulares, preenche dados ausentes e resolve inconsistências.

Seguindo o exemplo, depois de lavar, você corta os ingredientes no tamanho certo, mede as quantidades e os organiza em tigelas separadas para facilitar o cozimento, assim como ocorre na transformação dos dados. Sendo essa etapa responsável pela normalização dos dados (garantindo que os dados estejam na mesma escala), seleção de atributos (escolhendo os ingredientes certos para a receita) e discretização (transformando ingredientes contínuos em porções específicas).

Por fim, Se você tem muitos ingredientes ou quantidades excessivas, pode precisar diminuir a quantidade ou escolher os itens essenciais para garantir que o prato não fique sobrecarregado. Da mesma forma que ocorre na etapa de redução de dados, na qual você pode agregar informações, selecionar um subconjunto de atributos ou reduzir a dimensionalidade para tornar o processo mais eficiente.

Sendo assim, para a etapa de pré-processamento foram criadas sete funções, que são responsáveis pelo tratamento de valores nulos, exclusão de colunas com valores nulos ou zeros, tratamento de outliers, e normalização dos valores. Além disso, este notebook está organizado intercalando as funções (código) e suas respectivas explicações (texto). As explicações contém o nome da função, sua descrição, os parâmetros, o retorno da função e o detalhamento do código, para consultar a utilização dessas funções e seus resultados no projeto consultar [notebook principal](./main.ipynb) ou a [documentação](../documents/documentacao.md) do projeto.


#### 1. Importação das Bibliotecas


In [1]:
import numpy as np
import statistics as sts
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.preprocessing import LabelEncoder

#### 2. Funções de Pré-Processamento


**1. Função:** pre_processing

**Descrição:** Esta função é responsável por coordenar todo o processo de pré-processamento de um DataFrame. Ela aplica uma série de etapas para tratar valores nulos, remover colunas desnecessárias, tratar outliers e normalizar dados numéricos e categóricos.

**Parâmetros:**

- dataset: O DataFrame que será pré-processado.

**Retorno:**

- dataset: O DataFrame pré-processado.
- columns_trate: Lista de colunas tratadas para valores nulos.
- columns_drop_zero: Lista de colunas removidas por conterem apenas valores zero.
- columns_drop_null: Lista de colunas removidas por conterem apenas valores nulos.

**Detalhamento do Código:**

- `trate_null_value(dataset):` Trata valores nulos nas colunas numéricas e categóricas preenchendo com a mediana ou moda, respectivamente.
- `drop_columns_zero_values(dataset)`: Remove colunas que contêm apenas zeros.
- `drop_columns_null_values(dataset):` Remove colunas que contêm apenas valores nulos.
- `trate_outliers(dataset):` Substitui valores considerados outliers pela mediana da coluna.
- `normalize_numerics_columns(dataset):` Normaliza todas as colunas numéricas.
- `normalize_categoricals_columns(dataset):` Converte todas as colunas categóricas em valores numéricos utilizando codificação de rótulos.


In [2]:
def pre_processing(dataset):
    dataset, columns_trate = trate_null_value(dataset)  # Trata valores nulos nas colunas do DataFrame.
    dataset, columns_drop_zero = drop_columns_zero_values(dataset)  # Remove colunas que contêm apenas valores zero.
    dataset, columns_drop_null = drop_columns_null_values(dataset)  # Remove colunas que contêm apenas valores nulos.
    dataset = trate_outliers(dataset)  # Substitui outliers nas colunas numéricas pela mediana.
    dataset, label_encoders = normalize_categoricals_columns(dataset)  # Transforma colunas categóricas em valores numéricos.
    dataset, scalers = normalize_numerics_columns(dataset)  # Normaliza colunas numéricas para média 0 e desvio padrão 1.
    return dataset, columns_trate, columns_drop_zero, columns_drop_null, label_encoders, scalers

**2. Função:** trate_null_value

**Descrição**: Esta função identifica e trata valores nulos em um DataFrame preenchendo-os com a mediana para variáveis numéricas ou com a moda para variáveis categóricas.

**Parâmetros**:

- dataset: O DataFrame cujas colunas serão analisadas para valores nulos.

**Retorno:**

- dataset: O DataFrame com os valores nulos tratados.
- columns_trate: Lista de colunas que foram tratadas devido à presença de valores nulos.

**Detalhamento do Código:**

- `dataset[column].isnull().sum() > 0`: Verifica se há valores nulos na coluna.
- `np.issubdtype(dataset[column].dtype, np.number)`: Verifica se a coluna é do tipo numérico.
- `dataset[column].dropna()`: Remove valores nulos para cálculo estatístico.
- `sts.median(non_null_values)`: Calcula a mediana de valores não nulos.
- `dataset[column].fillna(median, inplace=True)`: Preenche valores nulos com a mediana.
- `dataset[column].mode()`: Calcula a moda para variáveis categóricas.
- `dataset[column].fillna(mode[0], inplace=True)`: Preenche valores nulos com a moda.


In [3]:
#Função que trata valores nulos  
def trate_null_value(dataset):
    columns_trate = []
    # Varre colunas do dataset recebido
    for column in dataset.columns:
        if dataset[column].isnull().sum() > 0:  # Verifica se há valores nulos na coluna
            if np.issubdtype(dataset[column].dtype, np.number):  # Verifica se a coluna é numérica
                non_null_values = dataset[column].dropna()  # Remove os NaN
                if len(non_null_values) > 0:  # Verifica se há dados não nulos suficientes
                    median = sts.median(non_null_values)  # Calcula a mediana
                    dataset[column].fillna(median, inplace=True) # Preenche com a mediana todos os valores nulos
                    columns_trate.append(f"Coluna '{column}' foi tratada, pois continha valores nulos.")
            else:  # Para colunas categóricas
                mode = dataset[column].mode() # Pega a moda da coluna
                if not mode.empty:  # Verifica se há uma moda disponível
                    dataset[column].fillna(mode[0], inplace=True)  # Preenche os NaN com a moda
                    columns_trate.append(f"Coluna '{column}' foi tratada, pois continha valores nulos.")

    return dataset, columns_trate

**3. Função:** drop_columns_zero_values

**Descrição**: Remove colunas que contêm apenas valores zero de um DataFrame, pois essas colunas não fornecem informações úteis.

**Parâmetros**:

- Dataset: O DataFrame cujas colunas serão analisadas para remoção.

**Retorno**:

- dataset: O DataFrame após a remoção das colunas.
- columns_drop: Lista de colunas que foram removidas por conterem apenas zeros.

**Detalhamento do Código:**

- `if (dataset[column] == 0).all()`: Verifica se todos os valores de uma coluna são zero.
- `dataset.drop(columns=[column], inplace=True)`: Remove a coluna se todos os valores forem zero.
- `columns_drop.append(f"Coluna '{column}'`: Adiciona em uma lista as colunas que foram removidas para log.


In [4]:
# Função para excluir colunas com valores que só contenham zeros
def drop_columns_zero_values(dataset):
    columns_drop = []
    # Itera sobre todas as colunas do DataFrame
    for column in dataset.columns:
        # Verifica se todos os valores da coluna são iguais a zero
        if (dataset[column] == 0).all():
            # Remove a coluna se todos os valores forem zero
            dataset.drop(columns=[column], inplace=True)
            # Adiciona em uma lista as colunas que foram removidas para log
            columns_drop.append(f"Coluna '{column}' foi removida, pois contém apenas zeros.")

    return dataset, columns_drop	

**4. Função:** drop_columns_null_values
**Descrição**: Remove colunas que contêm apenas valores nulos de um DataFrame, pois essas colunas não contribuem com dados úteis.

**Parâmetros**:

- dataset: O DataFrame cujas colunas serão analisadas para remoção.

**Retorno**:

- dataset: O DataFrame após a remoção das colunas.
- columns_drop: Lista de colunas que foram removidas por conterem apenas valores nulos.

**Detalhamento do Código**:

- `dataset[column].isnull().sum() == len(dataset[column])`: Verifica se todos os valores de uma coluna são nulos.
- `dataset.drop(columns=[column], inplace=True)`: Remove a coluna se todos os seus valores forem nulos.
- `columns_drop.append(f"Coluna '{column}'`: Adiciona em uma lista as colunas que foram removidas para log.


In [5]:
# Função para excluir colunas que só tenham valores nulos
def drop_columns_null_values(dataset):
    columns_drop = []
    for column in dataset.columns:
        if dataset[column].isnull().sum() > 0:  # Verifica se há valores nulos na coluna
            dataset.drop(columns=[column], inplace=True)  # Remove a coluna inteira
            # Adiciona em uma lista as colunas que foram removidas para log
            columns_drop.append(f"Coluna '{column}' excluída porque está completamente vazia.")
    return dataset, columns_drop

**5. Função**: trate_outliers

**Descrição**: Identifica e trata outliers nas colunas numéricas substituindo-os pela mediana da coluna, garantindo que outliers extremos não distorçam análises futuras.

**Parâmetros**:

- dataset: O DataFrame cujas colunas numéricas serão analisadas para outliers.

**Retorno**:

- dataset: O DataFrame com outliers tratados.

**Detalhamento do Código:**

- `np.nanmedian(dataset[column])`: Calcula a mediana ignorando valores nulos.
- `stats.zscore(dataset[column].dropna())`: Calcula os z-scores para identificar outliers.
- `dataset.loc[np.abs(zscores) > limiar, column] = median`: Substitui valores outliers pela mediana.


In [6]:
def trate_outliers(dataset):
    for column in dataset.columns:
        if np.issubdtype(dataset[column].dtype, np.number):  # Verifica se a coluna é numérica
            # Calcula a mediana ignorando os NaNs
            median = np.nanmedian(dataset[column])
            
            # Calcula os z-scores ignorando os NaNs
            zscores = stats.zscore(dataset[column].dropna())
            
            # Define um limiar de z-score (por exemplo, 3)
            limiar = 3
            
            # Reatribui valores maiores que o limiar para a mediana
            dataset.loc[np.abs(zscores) > limiar, column] = median
    
    return dataset

**6. Função:** normalize_numerics_columns

**Descrição:** Normaliza todas as colunas numéricas de um DataFrame utilizando o `StandardScaler`, padronizando os dados para uma média de 0 e desvio padrão de 1. Além de armazenar os objetos `StandardScaler` utilizados para cada coluna, permitindo que a normalização possa ser revertida posteriormente.

**Parâmetros:**

- `dataset`: O DataFrame cujas colunas numéricas serão normalizadas.

**Retorno:**

- `dataset`: O DataFrame com as colunas numéricas normalizadas.
- `scalers`: Um dicionário que armazena os objetos `StandardScaler` usados para cada coluna numérica.

**Detalhamento do Código:**

- `dataset.select_dtypes(include=[np.number]).columns`: Seleciona todas as colunas numéricas do DataFrame.
- `StandardScaler()`: Inicializa um objeto `StandardScaler`, que será utilizado para padronizar os dados.
- `scaler.fit_transform(dataset[[coluna]])`: Aplica a normalização à coluna numérica, convertendo os valores para uma escala com média 0 e desvio padrão 1.
- `scalers[coluna] = scaler`: Armazena o objeto `StandardScaler` correspondente à coluna processada.

In [7]:
# Função para normalizar colunas numéricas
def normalize_numerics_columns(dataset):
    scalers= {}
    numerics_columns = dataset.select_dtypes(include=[np.number]).columns
    for coluna in numerics_columns:
        scaler = StandardScaler()
        dataset[coluna] = scaler.fit_transform(dataset[[coluna]])
        scalers[coluna] = scaler  # Salva o scaler para reverter depois
    return dataset, scaler

**8. Função:** normalize_categoricals_columns

**Descrição:** Converte todas as colunas categóricas de um DataFrame em valores numéricos utilizando o `LabelEncoder`, transformando categorias em números inteiros.Além de armazenar os objetos `LabelEncoder` usados para cada coluna categórica.

**Parâmetros:**

- `dataset`: O DataFrame cujas colunas categóricas serão normalizadas.

**Retorno:**

- `dataset`: O DataFrame com as colunas categóricas convertidas em valores numéricos.
- `label_encoders`: Um dicionário que armazena os objetos `LabelEncoder` utilizados para cada coluna categórica.

**Detalhamento do Código:**

- `dataset.select_dtypes(include=['object']).columns`: Seleciona todas as colunas categóricas do DataFrame.
- `LabelEncoder()`: Inicializa um objeto `LabelEncoder`, que será utilizado para transformar os dados categóricos em números inteiros.
- `label_encoder.fit_transform(dataset[coluna])`: Aplica a transformação à coluna categórica, convertendo cada categoria em um valor numérico.
- `label_encoders[coluna] = label_encoder`: Armazena o objeto `LabelEncoder` correspondente à coluna processada.


In [8]:
# Função para normalizar colunas categóricas
def normalize_categoricals_columns(dataset):
    label_encoders = {}
    colunas_categoricas = dataset.select_dtypes(include=['object']).columns
    for coluna in colunas_categoricas:
        label_encoder = LabelEncoder()
        dataset[coluna] = label_encoder.fit_transform(dataset[coluna])
        label_encoders[coluna] = label_encoder  # Salva o LabelEncoder para reverter depois
    return dataset, label_encoders

**9. Função:** reverse_categoricals_columns

**Descrição:** Reverte a normalização de colunas categóricas em um DataFrame, convertendo os valores numéricos de volta para suas respectivas categorias originais usando o `LabelEncoder`.

**Parâmetros:**

- `dataset`: O DataFrame que contém as colunas categorizadas que foram normalizadas e precisam ser revertidas para seus valores originais.
- `label_encoders`: Um dicionário de `LabelEncoder` onde as chaves são os nomes das colunas e os valores são os objetos `LabelEncoder` que foram usados para codificar as colunas originais.

**Retorno:**

- `dataset`: O DataFrame com as colunas categóricas restauradas para seus valores originais.

**Detalhamento do Código:**

- `label_encoders.items()`: Itera sobre o dicionário que mapeia as colunas categóricas aos seus respectivos `LabelEncoder`.
- `if coluna in dataset.columns`: Verifica se a coluna categórica está presente no DataFrame.
- `pd.api.types.is_integer_dtype(dataset[coluna])`: Verifica se os dados da coluna estão no formato inteiro, o que é necessário para a transformação reversa.
- `dataset[coluna].astype(int)`: Converte os dados da coluna para inteiro caso não estejam no formato correto.
- `label_encoder.inverse_transform(dataset[coluna])`: Reverte a transformação numérica de volta para os valores categóricos originais usando o `LabelEncoder`.

In [9]:
# Função ajustada para reverter a normalização das colunas categóricas
def reverse_categoricals_columns(dataset, label_encoders):
    # Iterar sobre o dicionário de label_encoders e verificar se a coluna existe no dataset
    for coluna, label_encoder in label_encoders.items():
        if coluna in dataset.columns:
            # Verificar se os dados estão no formato correto para serem transformados
            if not pd.api.types.is_integer_dtype(dataset[coluna]):
                # Se a coluna não estiver em formato inteiro, tente convertê-la
                try:
                    dataset[coluna] = dataset[coluna].astype(int)
                except ValueError:
                    raise ValueError(f"Não foi possível converter a coluna {coluna} para inteiro.")
            # Reverter a codificação
            dataset[coluna] = label_encoder.inverse_transform(dataset[coluna])
    return dataset


**10. Função:** reverse_numerics_columns

**Descrição:** Reverte a normalização das colunas numéricas de um DataFrame, restaurando os valores normalizados para sua escala original, utilizando um dicionário de escaladores (`scalers`), que foram aplicados às colunas numéricas durante o processo de normalização.

**Parâmetros:**

- `dataset`: O DataFrame que contém as colunas numéricas que foram normalizadas e precisam ser revertidas para seus valores originais.
- `scalers`: Um dicionário onde as chaves são os nomes das colunas numéricas e os valores são os objetos `Scaler` (como `StandardScaler` ou `MinMaxScaler`) que foram utilizados para normalizar as colunas originais.

**Retorno:**

- `dataset`: O DataFrame com as colunas numéricas restauradas para seus valores originais.

**Detalhamento do Código:**

- `scalers.items()`: Itera sobre o dicionário que mapeia as colunas numéricas aos seus respectivos objetos `Scaler`.
- `scaler.inverse_transform(dataset[[coluna]])`: Aplica a função `inverse_transform` para reverter a normalização das colunas, restaurando seus valores à escala original.
- `dataset[[coluna]]`: Seleciona a coluna específica a ser transformada, garantindo que os dados estejam em formato de array 2D para o método `inverse_transform` do `Scaler`.

In [10]:
# Função para reverter a normalização das colunas numéricas
def reverse_numerics_columns(dataset, scalers):
    for coluna, scaler in scalers.items():
        dataset[coluna] = scaler.inverse_transform(dataset[[coluna]])
    return dataset

#### 3. Conclusão

Sendo assim, nesse notebook foi desenvolvido um conjunto de funções para facilitar o pré-processamento de dados. Essas funções têm como objetivo principal preparar os dados para análises mais profundas e para as modelagens preditivas, abordando o tratamento de valores nulos, a remoção de colunas irrelevantes, a normalização dos dados e o tratamento de outliers.

As funções para tratamento de valores nulos foram criadas para identificar e substituir valores ausentes nas colunas numéricas e categóricas. Para variáveis numéricas, os valores nulos são preenchidos pela mediana, enquanto para variáveis categóricas, a substituição é feita pela moda, assegurando que os dados estejam completos. Quanto às funções de remoção de colunas, estas foram desenvolvidas para excluir colunas que contenham apenas valores nulos ou apenas zeros, pois tais colunas não agregam valor às análises do projeto.

Somado a essas, as funções de normalização foram implementadas para garantir que os dados numéricos sejam padronizados, com média zero e desvio padrão de um, enquanto para as colunas categóricas, as funções transformam essas variáveis em valores numéricos, facilitando sua utilização em modelos quantitativos. Além disso, as funções de tratamento de outliers substituem valores extremos pela mediana da coluna, minimizando o impacto de valores atípicos nos resultados.

Em resumo, essas funções de pré-processamento foram desenvolvidas para serem reutilizáveis em diferentes conjuntos de dados, proporcionando um processo de manipulação de dados mais eficiente e padronizado. Ao encapsular a lógica de pré-processamento em funções, torna-se mais fácil aplicar técnicas de preparação de dados aos diferentes dataframes do projeto.

Por fim, para consultar as funções desenvolvidas para a exploração dos dados acesse o [notebook de exploração de dados](./data_exploration.ipynb), para visualizar a aplicação dessas funções no projeto acesse o [notebook principal](./main.ipynb), e para obter as análises dos resultados obtidos nessas etapas consulte a [documentação](../documents/documentacao.md).
